# Run ML trajectories

I want to see that the Mace potential uncertainties work as expected

In [1]:
import numpy as np
from pathlib import Path
import os
from glob import glob

%matplotlib inline
from matplotlib import pyplot as plt
from pathlib import Path
from ase.io import read
from ase import units
from ase.md.npt import NPT
from tqdm.auto import tqdm

from mace.calculators.foundations_models import mace_mp
from mace.calculators import MACECalculator

/home/mike/miniconda3/envs/cascade/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configuration

In [2]:
!ls ../0_setup/md/packmol-CH4-in-H2O=32-seed=3-blyp-npt=298/

md.log	md.traj


In [3]:
infer_device: str = 'cpu'
temperature = 298
steps: int = 512
initial_geometry = '../0_setup/final-geometries/packmol-CH4-in-H2O=32-seed=3-blyp.vasp'
reference_trajectory = '../0_setup/md/packmol-CH4-in-H2O=32-seed=3-blyp-npt=298/md.traj'

In [4]:
_initial_geometry = read(initial_geometry)
_reference_trajectory = read(reference_trajectory, index=':')

In [5]:
np.isclose(
    _reference_trajectory[0].get_positions(),
    _initial_geometry.positions
).mean()

0.0033003300330033004

### get ensemble members from directory

In [41]:
model_files = os.listdir('ensemble/')
model_files = [Path('ensemble')/f for f in model_files]#[:2]

MACECalculator can take multiple models in a list

In [42]:
model_files

[PosixPath('ensemble/model_2.pt'),
 PosixPath('ensemble/model_3.pt'),
 PosixPath('ensemble/model_0.pt'),
 PosixPath('ensemble/model_1.pt')]

In [43]:
calc = MACECalculator(model_paths=model_files, device='cpu')

Running committee mace with 4 models
No dtype selected, switching to float32 to match model dtype.


In [44]:
atoms = read(reference_trajectory, index='0')

In [45]:
atoms

Atoms(symbols='CH68O32', pbc=True, cell=[10.288817093428836, 10.288817093428836, 10.288817093428836], momenta=..., calculator=SinglePointCalculator(...))

In [46]:
atoms.get_positions().shape

(101, 3)

In [47]:
calc.calculate(atoms)

In [48]:
atoms.calc = calc

## Does the mean over the ensemble match?

In [49]:
atoms.get_potential_energy()

-15218.29296875

In [50]:
calc.get_potential_energy()

-15218.29296875

In [51]:
calc.get_potential_energies()

array([-15218.284, -15218.295, -15218.293, -15218.303], dtype=float32)

In [52]:
calc.get_potential_energies().mean()

-15218.293

In [53]:
calc.get_potential_energies().mean() == calc.get_potential_energy()

True

## Yes

## Do the variances match? 

In [54]:
atoms.calc.get_property('energy_var')

4.410743713378906e-05

In [55]:
calc.get_potential_energies().var(ddof=0)

4.4107437e-05

## no

In [56]:
atoms.positions[:3]

array([[ 4.71738309,  4.30004603,  4.48487714],
       [ 4.66705817, 10.18067382,  0.23975715],
       [ 9.96006038,  9.13071999,  6.28413929]])

In [57]:
type(calc.models)

list

In [58]:
len(calc.models)

4

In [59]:
calc.get_property('forces_comm').shape

(4, 101, 3)

In [65]:
calc.get_property('forces_comm').var(0).max()

6.064118e-06